In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain.llms import OpenAI

load_dotenv('.env')

True

In [1]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt"
res = requests.get(url)
with open("big.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./big.txt')
documents = loader.load()

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1147, which is longer than the specified 500
Created a chunk of size 1300, which is longer than the specified 500
Created a chunk of size 978, which is longer than the specified 500
Created a chunk of size 820, which is longer than the specified 500
Created a chunk of size 649, which is longer than the specified 500
Created a chunk of size 1126, which is longer than the specified 500
Created a chunk of size 1021, which is longer than the specified 500
Created a chunk of size 558, which is longer than the specified 500
Created a chunk of size 850, which is longer than the specified 500
Created a chunk of size 775, which is longer than the specified 500
Created a chunk of size 700, which is longer than the specified 500
Created a chunk of size 543, which is longer than the specified 500
Created a chunk of size 632, which is longer than the specified 500
Created a chunk of size 950, which is longer than the specified 500
Created a chunk of size 575, which is longer

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, embeddings)

c:\Users\Volkan\Desktop\LangChainDoc\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
retriever = vectorstore.as_retriever()

In [5]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What is the Document about?"
rag_chain.invoke(query)

c:\Users\Volkan\Desktop\LangChainDoc\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'The document is about the framing of the constitution. It includes chapters discussing the return of Michael Ivanovich with a letter, the old prince reading his manuscript, and a conversation about a printed sheet being necessary for the people.'